<a href="https://colab.research.google.com/github/pratheeshsailor/-Zee-Recommender-Systems/blob/main/Notebook_2_OTT_Rec_system_Case_Study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook 2

In [ ]:

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12,8)
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)


In [ ]:
movies = pd.read_fwf("zee-movies.dat",encoding="ISO-8859-1")
ratings =pd.read_fwf("zee-ratings.dat",encoding="ISO-8859-1")
users = pd.read_fwf("zee-users.dat",encoding="ISO-8859-1")

In [ ]:
delimiter ="::"

users = users["UserID::Gender::Age::Occupation::Zip-code"].str.split(delimiter,expand = True)
users.columns = ["UserID","Gender","Age","Occupation","Zipcode"]

users["Age"].replace({"1": "Under 18","18": "18-24","25": "25-34",
                          "35": "35-44","45": "45-49","50": "50-55","56": "56+"},inplace=True)

users["Occupation"] = users["Occupation"].astype(int).replace({0: "other",1: "academic/educator",2: "artist",
                                                               3: "clerical/admin",4: "college/grad student",
                                             5: "customer service",6: "doctor/health care",7: "executive/managerial",
                                             8: "farmer" ,9: "homemaker",10: "K-12 student",11: "lawyer",
                                             12: "programmer",13: "retired",14: "sales/marketing",15: "scientist",
                                             16: "self-employed",17: "technician/engineer",
                                             18: "tradesman/craftsman",19: "unemployed",20: "writer"},
                                            )

delimiter ="::"

ratings = ratings["UserID::MovieID::Rating::Timestamp"].str.split(delimiter,expand = True)
ratings.columns = ["UserID","MovieID","Rating","Timestamp"]


movies.drop(["Unnamed: 1","Unnamed: 2"],axis = 1,inplace=True)



delimiter ="::"

movies = movies["Movie ID::Title::Genres"].str.split(delimiter,expand = True)
movies.columns = ["MovieID","Title","Genres"]


movies.shape,ratings.shape,users.shape


((3883, 3), (1000209, 4), (6040, 5))

In [ ]:
movies["Release_year"] = movies["Title"].str.extract('^(.+)\s\(([0-9]*)\)$',expand = True)[1]
movies["Title"] = movies["Title"].str.split("(").apply(lambda x:x[0])

In [ ]:
from datetime import datetime
ratings["Watch_Hour"] =ratings["Timestamp"].apply(lambda x:datetime.fromtimestamp(int(x)).hour)
ratings.drop(["Timestamp"],axis = 1,inplace=True)

In [ ]:
movies.shape,ratings.shape,users.shape


((3883, 4), (1000209, 4), (6040, 5))

In [ ]:
df = users.merge(movies.merge(ratings,on="MovieID",how="outer"),on="UserID",how="outer")

In [ ]:
(df.isna().sum())/len(df)  * 100

UserID          0.017693
Gender          0.017693
Age             0.017693
Occupation      0.017693
Zipcode         0.017693
MovieID         0.000000
Title           0.000000
Genres          0.406443
Release_year    0.377854
Rating          0.017693
Watch_Hour      0.017693
dtype: float64

In [ ]:
data = df.copy()
data.dropna(inplace= True)

In [ ]:
data

,UserID,Gender,Age,Occupation,Zipcode,MovieID,Title,Genres,Release_year,Rating,Watch_Hour
0,1,F,Under 18,K-12 student,48067,1,Toy Story,Animation|Children's|Comedy,1995,5,23.0
1,1,F,Under 18,K-12 student,48067,48,Pocahontas,Animation|Children's|Musical|Romance,1995,5,23.0
2,1,F,Under 18,K-12 student,48067,150,Apollo 13,Drama,1995,5,22.0
3,1,F,Under 18,K-12 student,48067,260,Star Wars: Episode IV - A New Hope,Action|Adventure|Fantas,1977,4,22.0
4,1,F,Under 18,K-12 student,48067,527,Schindler's List,Drama|War,1993,5,23.0
...,...,...,...,...,...,...,...,...,...,...,...
1000204,6040,M,25-34,doctor/health care,11106,3683,Blood Simple,Drama|Film-Noir,1984,4,8.0
1000205,6040,M,25-34,doctor/health care,11106,3703,Mad Max 2,Action|Sci-Fi,1981,4,23.0
1000206,6040,M,25-34,doctor/health care,11106,3735,Serpico,Crime|Drama,1973,4,8.0
1000207,6040,M,25-34,doctor/health care,11106,3751,Chicken Run,Animation|Children's|Comedy,2000,4,23.0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 996144 entries, 0 to 1000208
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   UserID        996144 non-null  object 
 1   Gender        996144 non-null  object 
 2   Age           996144 non-null  object 
 3   Occupation    996144 non-null  object 
 4   Zipcode       996144 non-null  object 
 5   MovieID       996144 non-null  object 
 6   Title         996144 non-null  object 
 7   Genres        996144 non-null  object 
 8   Release_year  996144 non-null  object 
 9   Rating        996144 non-null  object 
 10  Watch_Hour    996144 non-null  float64
dtypes: float64(1), object(10)
memory usage: 91.2+ MB


In [ ]:
data.nunique()

UserID          6040
Gender             2
Age                7
Occupation        21
Zipcode         3439
MovieID         3682
Title           3633
Genres           358
Release_year      81
Rating             5
Watch_Hour        24
dtype: int64

In [ ]:
# 6040 unique UserID
# 7 different age groups
# 21 occupations
# 3493 different locations of users
# 3682 unique movies


In [ ]:
# There are movies available in database , which were never been watched by any user before .
# Thats is the reason we have lots of NaN values in our final dataset.

In [ ]:
data.shape

(996144, 11)

In [ ]:
m = movies[["MovieID","Title","Genres"]]


In [ ]:
m["Genres"] = m["Genres"].str.split("|")

In [ ]:

m = m.explode("Genres")
m["Genres"] = m["Genres"].replace({"Horro":"Horror","Sci-":"Sci-Fi","Sci":"Sci-Fi","Sci-F":"Sci-Fi","Dr":"Drama",
                     "Documenta":"Documentary","Wester":"Western","Fant":"Fantasy","Chil":"Children's","R":"Romance","D":"Drama","Rom":"Romance",
                     "Animati":"Animation","Childr":"Children's","Childre":"Children's","Fantas":"Fantasy","Come":"Comedy","Dram":"Drama","S":"Sci-Fi",
                     "Roma":"Romance","A":"Adventure","Children":"Children's","Adventu":"Adventure","Adv":"Adventure",
                      "Wa":"War","Thrille"  :"Thriller","Com":"Comedy","Comed":"Comedy","Acti":"Action",
                        "Advent":"Adventure","Adventur":"Adventure","Thri":"Thriller","Chi":"Children's","Ro":"Romance",
                        "F":"Fantasy","We":"Western","Documen":"Documentary",
                        "Music":"Musical","Children":"Children's" ,"Horr":"Horror","Children'":"Children's","Roman":"Romance","Docu":"Documentary","Th":"Thriller","Document":"Documentary"
                     })

In [ ]:
m

,MovieID,Title,Genres
0,1,Toy Story,Animation
0,1,Toy Story,Children's
0,1,Toy Story,Comedy
1,2,Jumanji,Adventure
1,2,Jumanji,Children's
...,...,...,...
3879,3949,Requiem for a Dream,Drama
3880,3950,Tigerland,Drama
3881,3951,Two Family House,Drama
3882,3952,"Contender, The",Drama


In [ ]:

m = pd.crosstab(m["MovieID"],       m["Genres"])
m = pd.DataFrame(np.where(m>=1,1,0),index = m.index,columns=m.columns)

In [ ]:
m

Genres,,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
MovieID,,,,,,,,,,,,,,,,,,,
1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
10,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
100,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
1000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1001,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
996,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
997,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0


In [ ]:
final_data = data.merge(m,on="MovieID",how="left").drop(["Genres"],axis = 1)

In [ ]:
final_data = pd.DataFrame(final_data)

## The movie with maximum no. of ratings is ___.



In [ ]:
final_data.groupby("Title")["Rating"].count().reset_index().sort_values(by="Rating",ascending=False).head(10)

,Title,Rating
125,American Beauty,3428
3092,Star Wars: Episode IV - A New Hope,2991
3093,Star Wars: Episode V - The Empire Strikes Back,2990
3094,Star Wars: Episode VI - Return of the Jedi,2883
1756,Jurassic Park,2672
2837,Saving Private Ryan,2653
3231,Terminator 2: Judgment Day,2649
2070,"Matrix, The",2590
255,Back to the Future,2583
2929,"Silence of the Lambs, The",2578


In [ ]:
final_data.sample(2)

,UserID,Gender,Age,Occupation,Zipcode,MovieID,Title,Release_year,Rating,Watch_Hour,,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
367779,2158,M,25-34,programmer,60618,2596,SLC Punk!,1998,5,7.0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
371645,2178,M,45-49,sales/marketing,85251,3879,"Art of War, The",2000,2,4.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
m = movies[["MovieID","Title","Genres"]]
m["Genres"] = m["Genres"].str.split("|")


In [ ]:
m = m.explode("Genres")
m["Genres"] = m["Genres"].replace({"Horro":"Horror",
                     "Sci-":"Sci-Fi","Sci":"Sci-Fi","Sci-F":"Sci-Fi",
                     "Dr":"Drama",
                     "Documenta":"Documentary",
                     "Wester":"Western",
                     "Fant":"Fantasy","Chil":"Children's","R":"Romance","D":"Drama","Rom":"Romance",
                     "Animati":"Animation",
                     "Childr":"Children's","Childre":"Children's",
                     "Fantas":"Fantasy","Come":"Comedy","Dram":"Drama","S":"Sci-Fi",
                     "Roma":"Romance","A":"Adventure","Children":"Children's",
                     "Adventu":"Adventure",
                      "Adv":"Adventure",
                      "Wa":"War",
                      "Thrille"  :"Thriller"     ,
                      "Com"    :"Comedy"   ,
                      "Comed"    :"Comedy",
                      "Acti"   :"Action",
                        "Advent"   :"Adventure",
                        "Adventur"      :"Adventure",
                        "Thri":"Thriller",
                        "Chi":"Children's",
                        "Ro":"Romance",
                        "F":"Fantasy",
                        "We":"Western",
                        "Documen":"Documentary"       ,
                        "Music":"Musical"         ,
                        "Children":"Children's" ,
                        "Horr":"Horror"          ,
                     "Children'":"Children's",
                     "Roman":"Romance","Docu":"Documentary","Th":"Thriller","Document":"Documentary"
                     })

In [ ]:
merged_data = ratings.merge(users,on="UserID",how="outer").merge(m,on="MovieID",how="outer")

In [ ]:
merged_data.groupby("Genres")[["Title","UserID"]].nunique()

,Title,UserID
Genres,,
,8,1344
Action,497,6011
Adventure,281,5894
Animation,104,4794
Children's,244,5280
Comedy,1182,6031
Crime,209,5662
Documentary,124,2237
Drama,1569,6037


## Number of Movie TItles been raten as per Genre by each type of use Occupation :

In [ ]:
Occupation_genre_count = merged_data.groupby(["Occupation","Genres"])["Title"].nunique().sort_values(ascending=False).reset_index()


In [ ]:
Occupation_genre_count.pivot(index="Occupation",columns="Genres",values="Title")

Genres,,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
Occupation,,,,,,,,,,,,,,,,,,,
K-12 student,5,409,234,97,229,855,135,35,852,62,30,228,94,74,313,210,349,111,43
academic/educator,6,458,262,96,231,1033,175,74,1239,62,40,282,104,94,417,241,435,129,58
artist,6,457,260,99,224,1002,176,69,1203,60,39,284,107,93,404,246,433,129,60
clerical/admin,6,440,245,98,212,969,169,56,1122,61,36,246,104,96,388,239,427,122,57
college/grad student,7,464,266,103,237,1072,185,83,1294,61,41,297,107,98,423,245,455,129,59
customer service,6,451,242,96,202,883,160,40,905,59,36,264,98,87,315,233,393,110,55
doctor/health care,7,450,253,102,228,946,165,68,1137,60,41,285,111,89,379,251,419,121,58
executive/managerial,6,466,269,101,234,1038,180,70,1243,62,40,301,106,94,416,254,438,130,62
farmer,5,322,191,81,151,519,98,8,473,44,21,113,57,43,182,181,233,70,25


In [ ]:
pd.set_option("max_colwidth", None)


# Co-occurance | Frequency Based Recommender System (Apriory)

In [ ]:
frame = data.groupby(["UserID","Title"])["Rating"].mean().unstack().reset_index().fillna(0).set_index('UserID')

In [ ]:
frame = (frame > 0).astype(int)
frame.shape

(6040, 3633)

In [ ]:
from mlxtend.frequent_patterns import apriori
frequent_itemsets_plus = apriori(frame, min_support=0.2,
                                 use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)


frequent_itemsets_plus['length'] = frequent_itemsets_plus['itemsets'].apply(lambda x: len(x))



/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [ ]:

frequent_itemsets_plus.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(1081, 3)

In [ ]:
from mlxtend.frequent_patterns import association_rules
rules = association_rules(frequent_itemsets_plus, metric ="lift", min_threshold = 0.8)
rules.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(4610, 10)

In [ ]:
rules.groupby(["antecedents"])["lift"].max().reset_index().merge(rules,on=["antecedents","lift"])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,lift,consequents,antecedent support,consequent support,support,confidence,leverage,conviction,zhangs_metric
0,(Star Wars: Episode V - The Empire Strikes Back ),1.959144,"(Star Wars: Episode VI - Return of the Jedi , Matrix, The , Jurassic Park , Star Wars: Episode IV - A New Hope )",0.495033,0.208609,0.202318,0.408696,0.099049,1.338381,0.969515
1,"(Matrix, The , Princess Bride, The )",1.760364,(Star Wars: Episode V - The Empire Strikes Back ),0.238245,0.495033,0.207616,0.871438,0.089677,3.927824,0.567027
2,"(Saving Private Ryan , Braveheart )",1.750430,"(Matrix, The )",0.276821,0.428808,0.207781,0.750598,0.089078,2.290247,0.592816
3,"(Sixth Sense, The , Star Wars: Episode IV - A New Hope )",1.714327,(Star Wars: Episode V - The Empire Strikes Back ),0.245033,0.495033,0.207947,0.848649,0.086648,3.336388,0.551919
4,"(Star Wars: Episode V - The Empire Strikes Back , Sixth Sense, The )",1.717235,(Star Wars: Episode IV - A New Hope ),0.244536,0.495199,0.207947,0.850372,0.086853,3.373717,0.552864
5,"(Die Hard , Star Wars: Episode IV - A New Hope )",1.853327,(Star Wars: Episode V - The Empire Strikes Back ),0.226656,0.495033,0.207947,0.917458,0.095745,6.117696,0.595375
6,"(Star Wars: Episode V - The Empire Strikes Back , Die Hard )",2.108770,(Raiders of the Lost Ark ),0.235596,0.416225,0.206788,0.877723,0.108727,4.774205,0.687843
7,"(E.T. the Extra-Terrestrial , Terminator 2: Judgment Day )",1.798631,(Star Wars: Episode V - The Empire Strikes Back ),0.234106,0.495033,0.208444,0.890382,0.092554,4.606601,0.579743
8,"(Star Wars: Episode VI - Return of the Jedi , Raiders of the Lost Ark , Matrix, The )",1.929708,(Star Wars: Episode V - The Empire Strikes Back ),0.218377,0.495033,0.208609,0.955269,0.100505,11.289006,0.616393
9,(Airplane! ),1.692648,(Back to the Future ),0.286589,0.427649,0.207450,0.723859,0.084891,2.072677,0.573596


In [ ]:
rules[rules["antecedents"] == rules.loc[4606]["antecedents"]].sort_values(by="lift",ascending=False).head(5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
3827,(American Beauty ),"(Sixth Sense, The , Being John Malkovich )",0.56755,0.230298,0.203146,0.357935,1.554224,0.072440,1.198791,0.824585
1761,(American Beauty ),(Election ),0.56755,0.251987,0.216887,0.382147,1.516536,0.073872,1.210665,0.787611
32,(American Beauty ),(Being John Malkovich ),0.56755,0.371026,0.311921,0.549592,1.481273,0.101345,1.396452,0.751312
4606,(American Beauty ),(Fight Club ),0.56755,0.240232,0.200000,0.352392,1.466884,0.063657,1.173192,0.735998
1926,(American Beauty ),"(Silence of the Lambs, The , Pulp Fiction )",0.56755,0.261093,0.215397,0.379522,1.453589,0.067214,1.190867,0.721581


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Item-Item Similarity Based Rec System

## Name the top 3 movies similar to ‘Liar Liar’ on the item-based approach.



In [ ]:
movies[movies["Title"].str.contains("Liar Liar")]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,MovieID,Title,Genres,Release_year
1455,1485,Liar Liar,Comedy,1997


In [ ]:
m = movies[["MovieID","Title","Genres"]]

m = m.explode("Genres")
m["Genres"] = m["Genres"].replace({"Horro":"Horror",
                     "Sci-":"Sci-Fi","Sci":"Sci-Fi","Sci-F":"Sci-Fi",
                     "Dr":"Drama",
                     "Documenta":"Documentary",
                     "Wester":"Western",
                     "Fant":"Fantasy","Chil":"Children's","R":"Romance","D":"Drama","Rom":"Romance",
                     "Animati":"Animation",
                     "Childr":"Children's","Childre":"Children's",
                     "Fantas":"Fantasy","Come":"Comedy","Dram":"Drama","S":"Sci-Fi",
                     "Roma":"Romance","A":"Adventure","Children":"Children's",
                     "Adventu":"Adventure",
                      "Adv":"Adventure",
                      "Wa":"War",
                      "Thrille"  :"Thriller"     ,
                      "Com"        :"Comedy"   ,
                      "Comed"         :"Comedy",
                      "Acti"          :"Action",
                        "Advent"        :"Adventure",
                        "Adventur"      :"Adventure",
                        "Thri":"Thriller",
                        "Chi":"Children's",
                        "Ro":"Romance",
                        "F":"Fantasy",
                        "We":"Western",
                        "Documen":"Documentary"       ,
                        "Music":"Musical"         ,
                        "Children":"Children's" ,
                        "Horr":"Horror"          ,
                     "Children'":"Children's",
                     "Roman":"Romance","Docu":"Documentary","Th":"Thriller","Document":"Documentary"
                     })
m = pd.crosstab(m["MovieID"],m["Genres"])
m = pd.DataFrame(np.where(m>=1,1,0),index = m.index,columns=m.columns)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def Hamming_distance(x1,x2):
  return np.sum(abs(x1-x2))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
Ranks = []
Query = "1485"
for candidate in m.index:
  if candidate == Query:
    continue
  Ranks.append([Query,candidate,Hamming_distance(m.loc[Query],m.loc[candidate])])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
Ranks = pd.DataFrame(Ranks,columns=["Query","Candidate","Hamming_distance"])
Ranks = Ranks.merge(movies[['MovieID', 'Title']], left_on='Query', right_on='MovieID').rename(columns={'Title': 'query_tittle'}).drop(columns=['MovieID'])
Ranks = Ranks.merge(movies[['MovieID', 'Title']], left_on='Candidate', right_on='MovieID').rename(columns={'Title': 'candidate_tittle'}).drop(columns=['MovieID'])
Ranks = Ranks.sort_values(by=['Query', 'Hamming_distance'])



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:

Ranks.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Query,Candidate,Hamming_distance,query_tittle,candidate_tittle
4,1485,1001,0,Liar Liar,"Associate, The"
5,1485,1002,0,Liar Liar,Ed's Next Move
13,1485,101,0,Liar Liar,Bottle Rocket
24,1485,102,0,Liar Liar,Mr. Wrong
25,1485,1020,0,Liar Liar,Cool Runnings


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Collaborative Filtering :

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
user_movie_ratings = ratings.pivot(index ="UserID",
              columns = "MovieID",
              values ="Rating").fillna(0)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
user_movie_ratings.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(6040, 3706)

In [ ]:
# 6040 users # 3706 movies

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
ratings.shape, users.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


((1000209, 4), (6040, 5))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:

rm_raw = ratings[['UserID', 'MovieID', 'Rating']].copy()
rm_raw.columns = ['UserId', 'ItemId', 'Rating']  # Lib requires specific column names


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
rm_raw.Rating = rm_raw.Rating.astype(int)
rm_raw.UserId = rm_raw.UserId.astype(int)
rm_raw.ItemId = rm_raw.ItemId.astype(int)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
rm_raw.nunique()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


UserId    6040
ItemId    3706
Rating       5
dtype: int64

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# !pip install cmfrec
!pip install cmfrec

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from cmfrec import CMF

model = CMF(k=5, lambda_=0.1, user_bias=False, item_bias=False, verbose=False)
model.fit(rm_raw)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Collective matrix factorization model
(explicit-feedback variant)


In [ ]:
rm_raw.shape,model.A_.shape,model.B_.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


((1000209, 3), (6040, 5), (3706, 5))

In [ ]:
model.A_.shape,model.B_.T.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


((6040, 5), (5, 3706))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
 model.topN(user=8, n=10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([2323,  296, 1136, 1089, 3421,  858, 1198, 3552,  260, 2776])

In [ ]:
 movies_to_recommend = model.topN(user=1, n=10)
 movies_to_recommend = movies_to_recommend[movies_to_recommend<3706]
 movies_to_recommend

 movies.MovieID = movies.MovieID.astype(int)
movies.loc[movies_to_recommend]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,MovieID,Title,Genres,Release_year
1421,1446,Kolya,Comedy,1996
2099,2168,Dance with Me,Drama|Romance,1998
2776,2845,White Boys,Drama,1999
2211,2280,Clay Pigeons,Crime,1998
1022,1035,"Sound of Music, The",Musical,1965
2018,2087,Peter Pan,Animation|Children's|Fantasy|Musical,1953
1951,2020,Dangerous Liaisons,Drama|Romance,1988
1028,1041,Secrets & Lies,Drama,1996
2035,2104,Tex,Drama,1982
595,599,"Wild Bunch, The",Western,1969


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
!pip install scikit-surprise

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
final_data= pd.DataFrame(final_data)
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import Reader

## The Reader class is used to parse a file containing ratings.It orders the data in format of (userid,title,rating) and even by considering the rating scale
reader = Reader(rating_scale=(0.5 , 5))
# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(final_data[["UserID","MovieID","Rating"]], reader) # loading the data as per the format

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
data

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
anti_set = data.build_full_trainset().build_anti_testset()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
trainset, testset = train_test_split(data, test_size=.15) # Splitting the data

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## User Based Collaborative Filtering :      


In [ ]:
algo = KNNWithMeans(k = 50, sim_options={'name': 'cosine', 'user_based': True})

# K value represents the (max) number of neighbors to take into account for aggregation. Example for every item it gives 50 nearest ones.
# There are many similarity options to calculate the similarity between the neighbors. Here, we have used the cosine similarity.
# when user_based = True then it performs user based collaborative filtering

algo.fit(trainset) #fitting the train dataset

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
# run the trained model against the testset
test_pred = algo.test(testset)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
test_pred[0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Prediction(uid='5964', iid='3591', r_ui=3.0, est=3.51163560643426, details={'actual_k': 50, 'was_impossible': False})

In [ ]:
# get RMSE on test set
print("User-based Model : Test Set")
accuracy.rmse(test_pred, verbose=True)

User-based Model : Test Set
RMSE: 0.9322


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.9322274375220012

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
accuracy.mae(test_pred, verbose=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


MAE:  0.7420


0.7419526407282924

In [ ]:
# we can query for specific predicions
uid = str(1)  # raw user id
iid = str(1)  # raw item id

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
pred = algo.predict(uid, iid, verbose=True)

user: 1          item: 1          r_ui = None   est = 4.89   {'actual_k': 50, 'was_impossible': False}


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
anti_pre = algo.test(anti_set)
pred_df = pd.DataFrame(anti_pre).merge(movies , left_on = ['iid'], right_on = ['MovieID'])
pred_df = pd.DataFrame(pred_df).merge(users , left_on = ['uid'], right_on = ['UserID'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Item Based Collaborative  Filtering :

In [ ]:
# K value represents the (max) number of neighbors to take into account for aggregation. Example for every item it gives 50 nearest ones.
# There are many similarity options to calculate the similarity between the neighbors . Here, we have used the cosine similarity.
# when user_based = False then it performs item based collaborative filtering

algo_i = KNNWithMeans(k=30, sim_options={'name': 'cosine', 'user_based': False})
algo_i.fit(trainset)

In [ ]:
test_pred = algo_i.test(testset)

In [ ]:
test_pred[0]

In [ ]:
# get RMSE on test set
print("Item-based Model : Test Set")
accuracy.rmse(test_pred, verbose=True)

In [ ]:
# we can query for specific predicions
uid = str(196)  # raw user id
iid = str(303)  # raw item id


In [ ]:
pred = algo_i.predict(uid, iid, verbose=True)

In [ ]:
# final_data[final_data["MovieID"]=="984"]

In [ ]:

tsr_inner_id = algo_i.trainset.to_inner_iid("1485") #Considering the movieId 1485 : Liar Liar

tsr_neighbors = algo_i.get_neighbors(tsr_inner_id, k=5) #Getting the 5 nearest neighbors for movieId 1


In [ ]:

movies[movies.MovieID.isin([algo.trainset.to_raw_iid(inner_id)
                       for inner_id in tsr_neighbors])] #Displaying the 5 nearest neighbors to the Liar Liar

# Matrix Factorisation:

In [ ]:
from surprise import SVD
from surprise.model_selection import cross_validate

In [ ]:
svd = SVD() #Suprise library uses the SVD algorithm to perform the matrix factorisation where as other libraries uses ALS
cross_validate(svd, data, measures=['rmse','mae'], cv = 5 , return_train_measures=True,verbose=True)
##The dataset is divided into train and test and with 5 folds the rmse has been calculated

In [ ]:

import pandas as pd

from surprise import Dataset

from surprise import SVD

from surprise import Reader

## The Reader class is used to parse a file containing ratings.It orders the data in format of (userid,title,rating) and even by considering the rating scale
reader = Reader(rating_scale=(0.5 , 5))
# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(final_data[["UserID","MovieID","Rating"]], reader) # loading the data as per the format

In [ ]:
svd = SVD(n_factors =10)
trainset = data.build_full_trainset()
svd.fit(trainset) ##Fitting the trainset with the help of svd

In [ ]:
svd.pu.shape , svd.qi.shape #pu gives the embeddings of Users and qi gives the embeddings of Items.

In [ ]:
#Storing all the movie titles in items
items = movies['Title'].unique()
##Considering the user '662'
test = [[662, iid, 4] for iid in items]
##Finding the user predictions(ratings) for all the movies
predictions = svd.test(test)
pred = pd.DataFrame(predictions)

In [ ]:
a = pred.sort_values(by='est', ascending=False) ##Sorting the values based on the estimated predictions
a[0:10] ##TOP 10

In [ ]:
testset = trainset.build_anti_testset()
predictions_svd = svd.test(testset) #Predicting for the test set

In [ ]:
from surprise import accuracy

In [ ]:
print('SVD - RMSE:', accuracy.rmse(predictions_svd, verbose=False))
print('SVD - MAE:', accuracy.mae(predictions_svd, verbose=False))